In [2]:
# import kagglehub
# custom_path = r"C:\Users\Tanuj Rajput\OneDrive\Desktop\tanujkaggle\12"
# path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# import shutil
# shutil.move(path, custom_path)

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("WELFake_Dataset.csv")

In [5]:
df = df.drop(columns=["Unnamed: 0"])

In [6]:
df['content'] = df['title'] + " " + df['text']

In [7]:
df.dropna(subset=['content', 'label'], inplace=True)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [9]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])

In [10]:
sequences = tokenizer.texts_to_sequences(df['content'])
padded = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [11]:
labels = df['label'].values

In [12]:
x_train, x_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout

In [14]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),  
    Dense(1, activation='sigmoid')
])

C:\Users\Tanuj Rajput\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 152s 165ms/step - accuracy: 0.7364 - loss: 0.5288 - val_accuracy: 0.8007 - val_loss: 0.5386
Epoch 2/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 187s 209ms/step - accuracy: 0.6976 - loss: 0.5951 - val_accuracy: 0.7939 - val_loss: 0.5208
Epoch 3/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 288s 322ms/step - accuracy: 0.8584 - loss: 0.3795 - val_accuracy: 0.8945 - val_loss: 0.3058
Epoch 4/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 159s 178ms/step - accuracy: 0.8848 - loss: 0.3130 - val_accuracy: 0.7581 - val_loss: 0.5240
Epoch 5/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 276s 308ms/step - accuracy: 0.8019 - loss: 0.4560 - val_accuracy: 0.9341 - val_loss: 0.2077
Epoch 6/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 303s 338ms/step - accuracy: 0.9429 - loss: 0.1704 - val_accuracy: 0.9408 - val_loss: 0.1624
Epoch 7/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 257s 287ms/step - accuracy: 0.9537 - loss: 0.1309 - val_accuracy: 0.9567 - val_loss: 0.1300
Epoch 8/50
895/895 ━━━━━━━━━━━━━━━━━━━━ 792s 885ms/step - accuracy: 0.9669 -

In [17]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test lossL {loss}, Test accuracy: {accuracy}")

448/448 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.9693 - loss: 0.2209
Test lossL 0.20511101186275482, Test accuracy: 0.9709253311157227


In [18]:
import pickle

In [19]:
model.save("fake_news_classifier_model.h5")

In [20]:
with open('tokenizer.pkl', 'wb') as f:      
    pickle.dump(tokenizer, f)
print("Model and tokenizer saved successfully.")

Model and tokenizer saved successfully.
